## Backpropagation (오차역전파)

## 5.4.1 곱셈 계층

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
    
        return out

    def backward(self, dout):
        dx = dout * self.y # x와y를 바꾼다.
        dy = dout * self.x 

        return dx, dy
    

In [2]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(int(price))

220


## 5.4.2 덧셈 계층

In [3]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [4]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()


# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

print(int(price))


# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
print("dall_price -> ", dall_price)
print("dtax -> ", dtax)

dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
print("dapple_price -> ", dapple_price)
print("dorange_price -> ", dorange_price)

dorange, dorange_num = mul_orange_layer.backward(dorange_price)
print("dorange -> ", int(dorange))
print("dorange_num -> ", int(dorange_num))

dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print("dapple -> ", dapple)
print("dapple_num -> ", int(dapple_num))

715
dall_price ->  1.1
dtax ->  650
dapple_price ->  1.1
dorange_price ->  1.1
dorange ->  3
dorange_num ->  165
dapple ->  2.2
dapple_num ->  110


## 5.5 활성화 함수 계층 구현하기

In [5]:
class Relu:

    def __init__(self):
        # T/F로 구성된 넘파이 배열
        # 순전파 입력인 x의 원소 값이 0 이하인 인덱스는 True.
        # 그 외 (0보다 큰 원소)는 False.
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx



In [6]:
import numpy as np

x = np.array( [[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = ( x <= 0 )
mask

[[ 1.  -0.5]
 [-2.   3. ]]


array([[False,  True],
       [ True, False]])

## 5.5.2 Sigmoid 계층

In [7]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout):
        # dσ(x)/dx = σ(x)⋅(1−σ(x))
        dx = dout * (1.0 - self.out) * self.out
        return dx

## 5.6 Affine/Softmax 계층 구현하기

In [8]:
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [9]:
dY = np.array([[1,2,3], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [10]:
# axis=0은 열 방향으로 합을 계산
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

In [11]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    # dout => 이전 레이어로부터 전달된 그래디언트(기울기)
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
    
        return dx


## 5.6.3 Softmax - with - Loss 계층

Softmax 계층은 입력 (a1,a2,a3)를 정규화하여 (y1,y2,y3)를 출력  <br>
Cross Entropy Error 계층은 Softmax의 출력 (y1,y2,y3)와 정답 레이블 (t1,t2,t3)를 받고, 이 데이터들로부터 손실 L을 출력

예를들어, 정답 레이블이 (0,1,0) 일 때 SoftMax 계층이 (0.3, 0.2, 0.5)를 출력 <br>
정답 레이블의 인덱스는 1 <br>
그런데 출력에서는 이때의 확률이 0.2에 불과 <br>
Softmax 계층의 역전파는 (0.3, -0.8, 0.5)라는 큰 오차를 전파 <br>  --> (0.3-0, 0.2-1, 0.5-0) <br>
결과적으로, Softmax 계층의 앞 계층들은 그 큰 오차로부터 더 크게 움직임 (학습 정도가 커짐)


In [12]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실
        self.y = None #softmax 출력
        self.t = None #정답 레이블 (ont-hot encoding)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        # 역전파 때는 전파하는 값을 배치의 수로 나눠 데이터 "1개당" 오차를 앞 계층으로 전파.
        dx = (self.y - self.t) / batch_size
        return dx

### 5.7.1 오차역전파법 구현하기

<b> 전제 </b> </br>
신경망에는 적응 가능한 가주치와 편향이 있다. <br>
가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 한다.<br>
신경망 학습은 4단계로 수행된다. <br>

<br>

<b> 1단계 - 미니배치 </b> </br>
훈련 데이터 중 일부를 무작위로 가져온다. (선별 데이터를 미니배치라고 함) </br>
해당 미니배치의 손실 함수 값을 줄이는 것이 목표 <br>

<b> 2단계 - 기울기 산출 </b> </br>
미니배치 손실 함수 값을 줄이기 위해 각 가중치 매개변수 기울기 구함 <br>
기울기는 손실 함수의 값을 가장 작게 하는 방향 제시 <br>

<b> 3단계 - 매개변수 갱신 </b> </br>
가중치 매개변수를 기울기 방향으로 아주 조금 갱신

<b> 4단계 - 반복 </b> </br>
1~3단계를 반복 

오차역전파법이 등장하는 단계는 두 번째인 '기울기 산출' 단계 <br>
오차역전파법 이용 시 느린 수치 미분과 달리 기울기를 효율적이고 빠르게 구할 수 있음 <br>

### 5.7.2 오차역전파법을 적용한 신경망 구현하기

2층 신경망을 TwoLayerNet 클래스로 구현

### TwoLayerNet 클래스의 인스턴스 변수


<b> params </b> - 딕셔너리 변수로, 신경망의 매개변수 보관 </br>
- params['W1']은 1번째 층의 가중치, params['b1']은 1번째 층의 편향 <br>
- params['W2']은 2번째 층의 가중치, params['b2']는 2번째 층의 편향 <br>

<br>


<b> layers </b> - 순서가 있는 딕셔너리 변수로, 신경망의 계층 보관 </br>
- layers['Affine1'], layers['Relu1'], layers['Affine2']와 같이 각 계층을 순서대로 유지

<br>

<b> lastLayer </b> - 신경망의 마지막 계층
- 이 예에서는 SoftmaWithLoss 계층

### TwoLayerNet 클래스의 메소드

<b> __init__(self, input_size, hidden_size, output_size, weight_init_std) </b> <br>
- 초기화 수행, 인수는 앞에서부터 입력층 뉴런 수, 은닉층 뉴런 수, 출력층 뉴런 수, 가중치 초기화 시 정규분포의 스케일


<b> predict(self,x) </b>
- 예측(추론)을 수행, 인수 x는 이미지 데이터

<b> loss(self,x,t) </b> </br>
- 손실 함수의 값을 구함, 인수 x는 이미지 데이터, t는 정답 레이블 <br>

<b> accuracy(self,x,t) </b> <br>
- 정확도를 구한다.

<b> numerical_gradient(self,x,t) </b> <br>
- 가중치 매개변수의 기울기를 수치 미분 방식을 구함 <br>

<b> gradient(self,x,t) </b> <br>
- 가중치 매개변수의 기울기를 오차역전파법으로 구함 

In [13]:
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [14]:
class TwoLayerNet:

    def __init__(self, input_size, hiden_szie, output_size,
                 weight_init_std=0.01):

        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size) # randn 함수를 사용하여 평균이 0이고 표준 편차가 1인 정규 분포에서 무작위로 수를 생성
                                                                     # 행렬의 크기는 input_size x hidden_size
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = \
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1) # 가장 높은 확률을 가진 클래스의 인덱스를 선택
        
        if t.ndim != 1 : t = np.argmax(t, axis=1) # t가 one-hot 인코딩되지 않았을 때, 최댓값의 인덱스로 변환하여 정답 클래스를 추출

        accuracy = np.sum(y == t) / float(x.shape[0]) # 예측된 클래스와 정답 클래스를 비교하여 일치하는 개수를 세고, 이를 전체 데이터 포인트 수로 나누어 정확도를 계산

        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    # 수치 미분
    def numerical_gradient(self, x, t):
        loss_W = lambda W : self.loss(x, t)

        grads = {} # 매개변수별로 수치 미분 결과를 저장
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    # 오차역전파
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

### 5.7.3 오차역전파법으로 구한 기울기 검증

기울기 구하는 2가지 방법 <br>
1. 수치 미분 <br>
2. 해석적으로 수식을 풀어 구하는 방법 <br>

후자인 해석적 방법은 오차역전파법 <br>

수치 미분은 느리다. 장점은 구현이 쉬움 <br>

오차역전파법을 제대로 구현했는지 검증하기 위해 수치 미분 사용 <br>
두 방식으로 구한 기울기가 거의 같음을 확인하는 작업을 기울기 확인(gradient check)이라고 함 <br>



In [15]:
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

In [16]:
# 데이터 읽기
(X_train, t_train), (x_test, t_test) = \
    load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

X_batch = X_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(X_batch, t_batch)
grad_backprop = network.gradient(X_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:1.9559642116786304e-10
b1:1.1096058585483406e-09
W2:7.198160315272096e-08
b2:1.434017297463619e-07


훈련 데이터 일부를 수치 미분으로 구한 기울기와 오차역전파법으로 구한 기울기의 오차 확인 <br>
각 가중치 매개변수의 차이의 절댓값을 구하고, 이를 평균한 값이 오차 <br>


### 5.7.4 오차역전파법을 사용한 학습 구현하기

In [17]:
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

In [18]:
# 데이터 읽기
(X_train, t_train), (X_test, t_tets) = \
    load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

iters_num = 10000
train_size = X_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    X_batch = X_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 오차역전파법으로 기울기 계산.
    grad = network.gradient(X_batch, t_batch)

    # 목적: 손실 함수를 최소화하는 방향으로 매개변수를 조정
    for key in ('W1', 'b1', 'W2', 'b2'):
        # 경사 하강법은 현재 매개변수에서 기울기를 빼는 것
        network.params[key] -= learning_rate * grad[key] # grad[key]는 해당 매개변수에 대한 기울기

    # 현재 미니배치에 대한 손실을 계산
    # 신경망이 현재 매개변수로 얼마나 좋은 예측을 하는지를 나타내는 값
    loss = network.loss(X_batch, t_batch)
    train_loss_list.append(loss)

    # 현재 반복 횟수 i가 에폭의 배수일 때 (즉, 에폭이 끝났을 때) 아래 코드 블록을 실행
    # 에폭(epoch)은 전체 훈련 데이터셋에 대해 한 번 학습을 완료한 상태
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(X_train, t_train)
        test_acc = network.accuracy(X_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.10441666666666667 0.1028
0.7877 0.7883
0.8765 0.8795
0.8979333333333334 0.9009
0.90785 0.9111
0.9149 0.9169
0.9194 0.9215
0.9250166666666667 0.9252
0.9278833333333333 0.9283
0.9308833333333333 0.9315
0.9346666666666666 0.9333
0.93735 0.938
0.9398833333333333 0.9395
0.9419833333333333 0.9408
0.9435 0.9422
0.94555 0.9438
0.9469333333333333 0.9443
